# Projeto 1 - Ciência dos Dados

Nome: Paulo Kim

Nome: Rodrigo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [ ]:
import os

filename = 'iPhone_X.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [ ]:
train = pd.read_excel(filename)
train = train.iloc[:,:2]
train.head(5)

In [ ]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test = test.iloc[:,:2]
test.head(5)

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nós pegamos como produto o iPhone X. Para sua classificação, dividimos os tweets em:

* Propaganda/oferta (0)
* Muito irrelevante (1)
* Irrelevante (2)
* Relevante (3)
* Muito relevante (4)

In [ ]:
# Função cleanup que realiza a filtragem de caracteres desnecessários
def cleanup(text):
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    text_subbed_list = text_subbed.split()
    
    for word in text_subbed_list:
        if '@' in word or word[:5] == 'https':
            text_subbed_list.remove(word)
        else:
            pass
    return ' '.join(text_subbed_list)  

In [ ]:
# Loop para realizar a limpeza
for i in range(len(train)):
    train.at[i,'Treinamento'] = cleanup(train.Treinamento[i]).lower() 
                                                                      
    
train.head(10)

In [ ]:
# Calculando o total de palavras no conjunto dos tweets 
total_palavras = 0 
for phrase in train.Treinamento:
    total_palavras += len(phrase.split())

___
### Montagens das séries e tabelas relativas para cada classificação

Filtragem do dataframe para cada classficação, para assim montar as tabelas com as probabilidades de cada palavra dada uma classificação

(TOKKI) Aqui não gostei muito do jeito que eu fiz, queria ter feito algo mais "limpo", pode mudar se tiver uma ideia melhor 

In [ ]:
# Transformando a lista de palavras da classficação Propaganda/Oferta (0) em uma série
lista_palavras_class_0 = []
palavras_class_0 = train.loc[train.Classificação == 0,:]

for phrase in palavras_class_0.Treinamento:
    lista_palavras_class_0 += phrase.split()
    
series_palavras_class_0 = pd.Series(lista_palavras_class_0)
series_palavras_class_0

In [ ]:
# Transformando a lista de palavras da classficação Muito Irrelevante (1) em uma série
lista_palavras_class_1 = []
palavras_class_1 = train.loc[train.Classificação == 1,:]

for phrase in palavras_class_1.Treinamento:
    lista_palavras_class_1 += phrase.split()
    
series_palavras_class_1 = pd.Series(lista_palavras_class_1)

In [ ]:
# Transformando a lista de palavras da classficação Irrelevante (2) em uma série
lista_palavras_class_2 = []
palavras_class_2 = train.loc[train.Classificação == 2,:]

for phrase in palavras_class_2.Treinamento:
    lista_palavras_class_2 += phrase.split()
    
series_palavras_class_2 = pd.Series(lista_palavras_class_2)

In [ ]:
# Transformando a lista de palavras da classficação Relevante (3) em uma série
lista_palavras_class_3 = []
palavras_class_3 = train.loc[train.Classificação == 3,:]

for phrase in palavras_class_3.Treinamento:
    lista_palavras_class_3 += phrase.split()
    
series_palavras_class_3 = pd.Series(lista_palavras_class_3)

In [ ]:
# Transformando a lista de palavras da classficação Muito Relevante (4) em uma série
lista_palavras_class_4 = []
palavras_class_4 = train.loc[train.Classificação == 4,:]

for phrase in palavras_class_4.Treinamento:
    lista_palavras_class_4 += phrase.split()
    
series_palavras_class_4 = pd.Series(lista_palavras_class_4)

In [ ]:
# Calculando a quantidade de cada palavra dada a classificação
tabela_palavra_class_0 = series_palavras_class_0.value_counts()
tabela_palavra_class_1 = series_palavras_class_1.value_counts()
tabela_palavra_class_2 = series_palavras_class_2.value_counts()
tabela_palavra_class_3 = series_palavras_class_3.value_counts()
tabela_palavra_class_4 = series_palavras_class_4.value_counts()

In [ ]:
print(tabela_palavra_class_1)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
# Calculando a probabilddiade de uma palavra pertencer a respectiva classificação   
P_0 = len(lista_palavras_class_0) / total_palavras
P_1 = len(lista_palavras_class_1) / total_palavras
P_2 = len(lista_palavras_class_2) / total_palavras
P_3 = len(lista_palavras_class_3) / total_palavras
P_4 = len(lista_palavras_class_4) / total_palavras

In [ ]:
# Smoothing
smoothing = {
    
    'V': 10**3,
    'alpha': 1
}

# Funções resposáveis em calcular as probabilidades com a aplicação do smoothing

def prob_frase_dado_class_0(frase):
    prob = 1
    for palavra in frase.split():
        try: # Verifica se a palavra está contida na série da respectiva categoria. 
            prob *= (tabela_palavra_class_0[palavra] + smoothing['alpha']) / (len(lista_palavras_class_0) + smoothing['alpha']*smoothing['V']) 
        except: # Caso a palavra não exista, a técnica de smoothing impede que a probabilidade resulte em zero
            prob *= smoothing['alpha'] / (len(lista_palavras_class_0) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_1(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_1[palavra] + smoothing['alpha']) / (len(lista_palavras_class_1) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_1) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_2(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_2[palavra] + smoothing['alpha']) / (len(lista_palavras_class_2) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_2) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_3(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_3[palavra] + smoothing['alpha']) / (len(lista_palavras_class_3) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_3) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_4(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_4[palavra] + smoothing['alpha']) / (len(lista_palavras_class_4) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_4) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

In [ ]:
def naive_bayes(tweet):
    p_0_dado_frase = prob_frase_dado_class_0(tweet) * P_0
    p_1_dado_frase = prob_frase_dado_class_1(tweet) * P_1
    p_2_dado_frase = prob_frase_dado_class_2(tweet) * P_2
    p_3_dado_frase = prob_frase_dado_class_3(tweet) * P_3
    p_4_dado_frase = prob_frase_dado_class_4(tweet) * P_4
    
    # As probabilidade são guardadas em uma lista, para assim, retornar o maior valor dela
    lista_de_probabilidades = [p_0_dado_frase,
                               p_1_dado_frase,
                               p_2_dado_frase,
                               p_3_dado_frase,
                               p_4_dado_frase]
    
    #Já que a lista está ordenada de acordo com as categorias, o index da maior probabilidade será a própria classificação
    return lista_de_probabilidades.index(max(lista_de_probabilidades))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
test.loc[:,'NBayes'] = 0
for i in range(len(test)):
    test.iloc[i,2] = naive_bayes(test.Teste[i])

In [ ]:
test

In [ ]:
test.Teste[721]

In [ ]:
cleanup(test.Teste[721])

In [ ]:
naive_bayes(cleanup(test.Teste[721]))

___
## Concluindo:

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**